In [ ]:
import scanpy as sc
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
import itertools as it
import json
import requests
import seaborn as sns
import pickle as pkl
from functools import reduce

import matplotlib as mpl
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [ ]:
sc.settings.verbosity = 4
sc.settings.set_figure_params(dpi=80)
print(sc.__version__)
sc.settings.n_jobs=12

In [ ]:
prefix = '/data/codec/production.run/mrna/'

In [ ]:
path = prefix + 'pkls/aggr/tnk.2.pkl'

# with open(path,'wb') as file:
#     pkl.dump(tnk, file, protocol=4)

with open(path,'rb') as file:
    tnk = pkl.load(file)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
for color, ax, palette in zip(['cond', 'leiden'], ax, [None, sc.pl.palettes.default_20]):
    sc.pl.umap(tnk,color=color, ax=ax, show=False, return_fig=False, size=2, palette=palette)

In [ ]:
clusts = [6, 10]

In [ ]:
sub_tnk = dict()
for clust in clusts:
    sub_tnk[str(clust)] = tnk[tnk.obs['leiden'] == str(clust)].copy()

In [ ]:
sc.tl.leiden(sub_tnk['6'], resolution=0.1) # subcluster them using Leiden
sc.pl.umap(sub_tnk['6'],color='leiden', size=0.6)

In [ ]:
sc.tl.leiden(sub_tnk['10'], resolution=0.2) # subcluster them using Leiden
sc.pl.umap(sub_tnk['10'],color='leiden', size=0.6)

In [ ]:
groupings = [[0, 2],
             ]
grouped_clusts = [i for j in groupings for i in j]
numclusts = np.unique(sub_tnk['10'].obs['leiden'].values.astype(int))
for i in np.setdiff1d(numclusts, grouped_clusts):
    groupings.append([i])

In [ ]:
ctdict = dict()
for i in range(len(groupings)):
    ctdict['ct%s' % str(i)] = groupings[i]

sub_tnk['10'].obs['celltype'] = sub_tnk['10'].obs['leiden']
for ct in ctdict:
    for clust in ctdict[ct]:
        sub_tnk['10'].obs['celltype'].replace(str(clust), ct, regex=True, inplace=True)
sub_tnk['10'].obs['leiden'] = [i.strip('ct') for i in sub_tnk['10'].obs['celltype'].astype('category')]

In [ ]:
sc.pl.umap(sub_tnk['10'],color=['leiden'], size=3)

In [ ]:
def sub_cluster_mapper(adata, sub_adatas):
    '''
    This takes in the adata object and inserts a new leiden column in the `.obs`. 
    
    This function is really convoluted and there's probably a better, simpler way to do it,
    but it should theoretically work for any number of subclusters
    '''
    # ideally you'd make a copy of the adata object here, so we don't have to change the original
    # this would be in case we want to run it multiple times, perhaps the resolutions we put in didn't subset the clusters like we had hoped
    # and we need to run multiple times to adjust the resolution slightly
    
    # this block is to figure out that there are two new subclusters and they should be named 8, 9
    total_new_clusts = 0
    old_clusts = sub_adatas.keys()
    for sub_adata in sub_adatas:
        total_new_clusts += sub_adatas[sub_adata].obs['leiden'].astype(int).unique().max() + 1
    total_added_clusts = total_new_clusts - len(sub_adatas)
    new_clust_names_start = max(adata.obs['leiden'].astype(int))+1
    new_added_clust_names = [str(i) for i in range(new_clust_names_start,
                                                   new_clust_names_start + total_added_clusts)]
    
    # this block is to build a new list of leiden clusters from the old one 
    new_leiden = list()
    leiden_col = adata.obs['leiden'].copy()

    # this builds the new leiden cluster list, now adding a .1, .2, etc. to each new cluster
    for obs in leiden_col.index:
        clust_name = leiden_col.loc[obs]
        if clust_name not in old_clusts or sub_adatas[clust_name].obs.loc[obs, 'leiden'] == '0':
            new_leiden.append(clust_name)
        else:
            new_leiden.append(clust_name + '.%s' % sub_adatas[clust_name].obs.loc[obs,'leiden'])

    # this renames the .1, .2, etc clusters to the new, better names I came up with above (8 and 9)
    new_leiden = pd.Series(new_leiden, index=adata.obs_names)
    added_clusts = np.setdiff1d(new_leiden,adata.obs['leiden'])
    new_leiden.replace(dict(zip(added_clusts, new_added_clust_names)), inplace=True)
    
    # replace the old leiden column, must do these steps sequentially 
    adata.obs['leiden'] = new_leiden.astype(int) # to order the clusters by number
    adata.obs['leiden'] = new_leiden.astype(str) # to convert to string as normal
#     adata.obs['leiden'] = new_leiden.astype('category') # don't do this, it messes things up, just let scanpy do it as it plots
    return adata

In [ ]:
tnk = sub_cluster_mapper(tnk, sub_tnk)

In [ ]:
sc.pl.umap(tnk,color=['cond', 'leiden'], size=3)

In [ ]:
groupings = [[1, 3, 4, 7, 8, 11],
             [0, 5, 9],
             [2, 10]
             ]
grouped_clusts = [i for j in groupings for i in j]
numclusts = np.unique(tnk.obs['leiden'].values.astype(int))
for i in np.setdiff1d(numclusts, grouped_clusts):
    groupings.append([i])

In [ ]:
ctdict = dict()
for i in range(len(groupings)):
    ctdict['ct%s' % str(i)] = groupings[i]

tnk.obs['celltype'] = tnk.obs['leiden']
for ct in ctdict:
    for clust in ctdict[ct]:
        tnk.obs['celltype'].replace(str(clust), ct, regex=True, inplace=True)
tnk.obs['leiden'] = [i.strip('ct') for i in tnk.obs['celltype'].astype('category')]

In [ ]:
sc.pl.umap(tnk, color=['leiden', 'CD3|CD3E','CD4|CD4','CD8|CD8A','CD56|NCAM1','CD16|FCGR3A'], size=3)

In [ ]:
sc.settings.verbosity = 0 
warnings.filterwarnings('ignore')
sc.tl.rank_genes_groups(tnk, groupby='leiden', n_genes=20, groups=['4','5'])
warnings.filterwarnings('default')
sc.pl.rank_genes_groups(tnk, ncols=5, n_genes=20)
sc.settings.verbosity = 4

Cluster 4 is probably B/T multiplets and 5 is probably T/M multiplets.

In [ ]:
for ct, clust in zip(['acg.t','br.t','p.t','acg.nk','p.nk','br.nk'],
                     [0, 1, 2, 3, 6, 7]
                    ):
    path = prefix + 'obs/cts/%s.txt' % ct

    with open(path,'w') as file:
        for bc in tnk.obs.index[tnk.obs['leiden'] == str(clust)]:
            file.write(bc + '\n')

# DELETE THE REST AFTER I'VE CONFIRMED I DON'T NEED ANYTHING FROM IT

In [ ]:
acg_t = tnk[tnk.obs['leiden'] == '0'].copy()
# acg_nk = tnk[tnk.obs['leiden'] == '3'].copy()
# br_t = tnk[tnk.obs['leiden'] == '1'].copy()
# br_nk = tnk[tnk.obs['leiden'] == '7'].copy()
# p_t = tnk[tnk.obs['leiden'] == '2'].copy()
# p_nk = tnk[tnk.obs['leiden'] == '6'].copy()

# Cell Type Split

In [ ]:
total_pcs = 75

In [ ]:
warnings.filterwarnings('ignore')
sc.settings.n_jobs = 12
sc.pp.pca(acg_t,n_comps=total_pcs)
warnings.filterwarnings('default')

In [ ]:
sc.pl.pca_variance_ratio(acg_t,log=True, n_pcs=total_pcs)

In [ ]:
df_loadings = pd.DataFrame(acg_t.varm['PCs'], index=acg_t.var_names)
df_rankings = pd.DataFrame((-1 * df_loadings.values).argsort(0).argsort(0), index=df_loadings.index, columns=df_loadings.columns)

In [ ]:
num = 50
percent_ribos = list()
for pc in range(total_pcs):
    top_genes = df_loadings[pc].sort_values(ascending=True)[:num].index
    percent_ribos.append(len([i for i in top_genes if i.startswith('RP')])/num)

In [ ]:
plt.bar(range(len(percent_ribos)), percent_ribos);

In [ ]:
acg_t.obs['ribo_pc'] = acg_t.obsm['X_pca'][:,0]

In [ ]:
warnings.filterwarnings('ignore')
sc.pp.regress_out(acg_t, 'ribo_pc', n_jobs=1)
warnings.filterwarnings('default')

In [ ]:
total_pcs = 75

In [ ]:
warnings.filterwarnings('ignore')
sc.settings.n_jobs = 12
sc.pp.pca(acg_t,n_comps=total_pcs)
warnings.filterwarnings('default')

In [ ]:
sc.pl.pca_variance_ratio(acg_t,log=True, n_pcs=total_pcs)

In [ ]:
df_loadings = pd.DataFrame(acg_t.varm['PCs'], index=acg_t.var_names)
df_rankings = pd.DataFrame((-1 * df_loadings.values).argsort(0).argsort(0), index=df_loadings.index, columns=df_loadings.columns)

In [ ]:
num = 50
percent_ribos = list()
for pc in range(total_pcs):
    top_genes = df_loadings[pc].sort_values(ascending=True)[:num].index
    percent_ribos.append(len([i for i in top_genes if i.startswith('RP')])/num)

In [ ]:
plt.bar(range(len(percent_ribos)), percent_ribos);

In [ ]:
warnings.filterwarnings('ignore')
sc.pp.neighbors(acg_t, n_neighbors=15, n_pcs=40)
warnings.filterwarnings('default')

In [ ]:
sc.tl.umap(acg_t)

In [ ]:
sc.tl.leiden(acg_t, resolution=0.5)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
for color, ax, palette in zip(['cond', 'leiden'], ax, [None, sc.pl.palettes.default_20]):
    sc.pl.umap(acg_t,color=color, ax=ax, show=False, return_fig=False, size=4, palette=palette)

In [ ]:
sc.settings.verbosity = 0
warnings.filterwarnings('ignore')
sc.tl.rank_genes_groups(acg_t, groupby='leiden', n_genes=20)
warnings.filterwarnings('default')
sc.pl.rank_genes_groups(acg_t, ncols=5, n_genes=20)
sc.settings.verbosity = 4

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(30,15))
for val, ax in tqdm(zip(acg_t.obs['leiden'].dtype.categories, np.ravel(ax))):
    acg_t.obs['val'] = acg_t.obs['leiden'] == val
    ax.set_facecolor('black')
    sc.pl.umap(acg_t,color='val', ax=ax, show=False, return_fig=False, title=val)
    acg_t.obs.drop(columns='val', inplace=True)
plt.tight_layout()

In [ ]:
clusts = [1, 2, 3, 4, 5, 6, 7]

In [ ]:
sub_acgt = dict()
for clust in clusts:
    sub_acgt[str(clust)] = acg_t[acg_t.obs['leiden'] == str(clust)].copy()

In [ ]:
sc.tl.leiden(sub_acgt['1'], resolution=0.7) # subcluster them using Leiden
sc.pl.umap(sub_acgt['1'],color='leiden', size=3)

In [ ]:
groupings = [[2, 4],
            ]
grouped_clusts = [i for j in groupings for i in j]
numclusts = np.unique(sub_acgt['1'].obs['leiden'].values.astype(int))
for i in np.setdiff1d(numclusts, grouped_clusts):
    groupings.append([i])

In [ ]:
ctdict = dict()
for i in range(len(groupings)):
    ctdict['ct%s' % str(i)] = groupings[i]

sub_acgt['1'].obs['celltype'] = sub_acgt['1'].obs['leiden']
for ct in ctdict:
    for clust in ctdict[ct]:
        sub_acgt['1'].obs['celltype'].replace(str(clust), ct, regex=True, inplace=True)
sub_acgt['1'].obs['leiden'] = [i.strip('ct') for i in sub_acgt['1'].obs['celltype'].astype('category')]

In [ ]:
sc.pl.umap(sub_acgt['1'],color='leiden', size=5)

In [ ]:
sc.tl.leiden(sub_acgt['2'], resolution=0.2) # subcluster them using Leiden
sc.pl.umap(sub_acgt['2'],color='leiden', size=5)

In [ ]:
sc.tl.leiden(sub_acgt['3'], resolution=0.3) # subcluster them using Leiden
sc.pl.umap(sub_acgt['3'],color='leiden', size=5)

In [ ]:
sc.tl.leiden(sub_acgt['4'], resolution=0.3) # subcluster them using Leiden
sc.pl.umap(sub_acgt['4'],color='leiden', size=5)

In [ ]:
sc.tl.leiden(sub_acgt['5'], resolution=0.3) # subcluster them using Leiden
sc.pl.umap(sub_acgt['5'],color='leiden', size=5)

In [ ]:
sc.tl.leiden(sub_acgt['6'], resolution=0.3) # subcluster them using Leiden
sc.pl.umap(sub_acgt['6'],color='leiden', size=5)

In [ ]:
sc.tl.leiden(sub_acgt['7'], resolution=0.3) # subcluster them using Leiden
sc.pl.umap(sub_acgt['7'],color='leiden', size=5)

In [ ]:
def sub_cluster_mapper(adata, sub_adatas):
    '''
    This takes in the adata object and inserts a new leiden column in the `.obs`. 
    
    This function is really convoluted and there's probably a better, simpler way to do it,
    but it should theoretically work for any number of subclusters
    '''
    # ideally you'd make a copy of the adata object here, so we don't have to change the original
    # this would be in case we want to run it multiple times, perhaps the resolutions we put in didn't subset the clusters like we had hoped
    # and we need to run multiple times to adjust the resolution slightly
    
    # this block is to figure out that there are two new subclusters and they should be named 8, 9
    total_new_clusts = 0
    old_clusts = sub_adatas.keys()
    for sub_adata in sub_adatas:
        total_new_clusts += sub_adatas[sub_adata].obs['leiden'].astype(int).unique().max() + 1
    total_added_clusts = total_new_clusts - len(sub_adatas)
    new_clust_names_start = max(adata.obs['leiden'].astype(int))+1
    new_added_clust_names = [str(i) for i in range(new_clust_names_start,
                                                   new_clust_names_start + total_added_clusts)]
    
    # this block is to build a new list of leiden clusters from the old one 
    new_leiden = list()
    leiden_col = adata.obs['leiden'].copy()

    # this builds the new leiden cluster list, now adding a .1, .2, etc. to each new cluster
    for obs in leiden_col.index:
        clust_name = leiden_col.loc[obs]
        if clust_name not in old_clusts or sub_adatas[clust_name].obs.loc[obs, 'leiden'] == '0':
            new_leiden.append(clust_name)
        else:
            new_leiden.append(clust_name + '.%s' % sub_adatas[clust_name].obs.loc[obs,'leiden'])

    # this renames the .1, .2, etc clusters to the new, better names I came up with above (8 and 9)
    new_leiden = pd.Series(new_leiden, index=adata.obs_names)
    added_clusts = np.setdiff1d(new_leiden,adata.obs['leiden'])
    new_leiden.replace(dict(zip(added_clusts, new_added_clust_names)), inplace=True)
    
    # replace the old leiden column, must do these steps sequentially 
    adata.obs['leiden'] = new_leiden.astype(int) # to order the clusters by number
    adata.obs['leiden'] = new_leiden.astype(str) # to convert to string as normal
#     adata.obs['leiden'] = new_leiden.astype('category') # don't do this, it messes things up, just let scanpy do it as it plots
    return adata

In [ ]:
acg_t = sub_cluster_mapper(acg_t, sub_acgt)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
for color, ax, palette in zip(['cond', 'leiden'], ax, [None, sc.pl.palettes.default_20]):
    sc.pl.umap(acg_t,color=color, ax=ax, show=False, return_fig=False, size=4, palette=palette)

In [ ]:
sc.settings.verbosity = 0
warnings.filterwarnings('ignore')
sc.tl.rank_genes_groups(acg_t, groupby='leiden', n_genes=1000, use_raw=True)
warnings.filterwarnings('default')
sc.pl.rank_genes_groups(acg_t, ncols=5, n_genes=20)
sc.settings.verbosity = 4

In [ ]:
sc.pl.umap(acg_t, color=['CD4|CD4','CD8|CD8A'], size=2, ncols=5, use_raw=True) 

In [ ]:
sc.pl.umap(acg_t, color=['CCR7', 'CD45RA|PTPRC','CD8|CD8A','CD8A', 'CD8B'], size=2, ncols=5, use_raw=True) 

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.set_facecolor('black')
sc.pl.umap(acg_t, color='leiden', size=10, ax=ax)

In [ ]:
path = prefix + 'pkls/aggr/tnk.3.pkl'

with open(path,'wb') as file:
    pkl.dump(acg_t, file, protocol=4)

# with open(path,'rb') as file:
#     acg_t = pkl.load(file)

In [ ]:
def add_raw(adata, transformed=True):
    path = prefix + 'pkls/aggr/wells.sng.w_covars.pkl'
    
    with open(path,'rb') as file:
        wells = pkl.load(file)
    
    for well in wells:
        wells[well]['adata'].obs_names = [i[:16] + '-%s' % well for i in wells[well]['adata'].obs_names]
    
    raw = wells[0]['adata'].concatenate(*[wells[i]['adata'] for  i in range(1, 12)])[adata.obs_names,:]
    
    if transformed == True:
        sc.pp.normalize_per_cell(raw,counts_per_cell_after=1e6)
        sc.pp.log1p(raw)
    
    adata.raw = raw
    return

In [ ]:
add_raw(acg_t)

In [ ]:
raw_transcripts = acg_t.raw.var_names
transcripts = acg_t.var_names
proteins = [i for i in acg_t.obs.columns if '|' in i]

In [ ]:
name = 'CCR7'
print([i for i in proteins if name in i])
print([i for i in transcripts if name in i])
print([i for i in raw_transcripts if name in i])

In [ ]:
proteins.index('TCRgd|TRD_TRG') 
# proteins.index('CD94|KLRD1')

In [ ]:
features = ['CD3|CD3E', 'CD4|CD4','CD8|CD8A', 'CD45RA|PTPRC','CD45RO|PTPRC',
            'CD185|CXCR5', 'CD62L|SELL', 'CXCR3','CCR4', 'CCR6', 
            'LAT','RGS10','CD69|CD69', 'PITPNC1', 'SLC5A3',
            'TRDC', 'TRDV2', 'IKZF2', 'FOXP3', 'CD94|KLRD1',
            'CD26|DPP4', 'CD103|ITGAE', 'CCR7', 'NKG7','CCL5'
           ]

In [ ]:
fig, ax = plt.subplots(5,5,figsize=(30,27.5))
for ax, f in zip(np.ravel(ax), features):
    ax.set_facecolor('black')
    sc.pl.umap(acg_t, color=f, ax=ax,show=False, return_fig=False, size=5, use_raw=True)

In [ ]:
cts = ["Naïve CD4", "Th2", "Early Activated CD4", "Naïve CD8", 
       "Effector Memory RA CD8", "Effector Memory CD8", "Resting Treg", "Mito", 
       "Th17", "Tfh", "Tfh/Proliferation", "Doublet? 1", 
       "R848 Contam 1", "Early Activated CD8", "Central Memory CD8", "Gamma Delta Treg?", 
       "Gamma Delta Type2", "R848 Contam? 2", "Activated Treg", "Doublet? 2"]

In [ ]:
ctdict = dict(zip(cts, [[i] for i in range(20)]))

In [ ]:
acg_t.obs['ct1'] = acg_t.obs['leiden']
for ct in ctdict:
    for clust in ctdict[ct]:
        acg_t.obs['ct1'].replace(str(clust), ct, regex=True, inplace=True)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.set_facecolor('whitesmoke')
sc.pl.umap(acg_t, color='ct1', size=10, ax=ax, legend_loc='on data', legend_fontsize=10)

In [ ]:
len(acg_t[(acg_t.obs['ct1'] == 'Gamma Delta Type2') & (acg_t.obs['cond'] == 'C')])/len(acg_t[acg_t.obs['cond'] == 'C'])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.set_facecolor('whitesmoke')
sc.pl.umap(acg_t, color='ct1', size=10, ax=ax)

In [ ]:
fig, ax = plt.subplots(1,7, figsize=(30,4))
for val, ax in tqdm(zip(acg_t.obs['cond'].dtype.categories, np.ravel(ax))):
    acg_t.obs['val'] = acg_t.obs['cond'] == val
    ax.set_facecolor('black')
    sc.pl.umap(acg_t,color='val', ax=ax, show=False, return_fig=False, title=val)
    acg_t.obs.drop(columns='val', inplace=True)
plt.tight_layout()

In [ ]:
sc.settings.verbosity = 0
warnings.filterwarnings('ignore')
sc.tl.rank_genes_groups(acg_t_subs['Naive CD8'], groupby='cond', n_genes=100, groups=['A'], reference='C')
warnings.filterwarnings('default')
sc.pl.rank_genes_groups(acg_t_subs['Naive CD8'])
sc.settings.verbosity = 4

In [ ]:
good_cts = ["Naïve CD4", "Th2", "Early Activated CD4", "Naïve CD8", 
            "Effector Memory RA CD8", "Effector Memory CD8", "Resting Treg", 
            "Th17", "Tfh", "Tfh/Proliferation", "Early Activated CD8", 
            "Central Memory CD8", "Gamma Delta Treg?", 
            "Gamma Delta Type2", "Activated Treg"
           ]

In [ ]:
sc.settings.verbosity = 0
warnings.filterwarnings('ignore')
fig, ax = plt.subplots(3, 5, figsize=(30,18))
for ct, ax in tqdm(zip(good_cts,np.ravel(ax))):
    rank_adata = acg_t[acg_t.obs['ct1'] == ct].copy()
    sc.tl.rank_genes_groups(rank_adata, groupby='cond', n_genes=20, groups=['A'], reference='C')
    x = range(20)
    y = [i[0] for i in rank_adata.uns['rank_genes_groups']['scores'].tolist()]
    txts = [i[0] for i in rank_adata.uns['rank_genes_groups']['names'].tolist()]
    ax.scatter(x, y, s=0)
    for i, txt in enumerate(txts):
        ax.annotate(txt, (x[i], y[i]), rotation=45, size=10)
    ax.set_title(ct)

In [ ]:
gene = 'CD137|TNFRSF9'
fig, ax = plt.subplots(1,2,figsize=(12,5))
for cond, ax in zip(['C','A'], ax):
    ax.set_facecolor('black')
    sc.pl.umap(acg_t[acg_t.obs['cond'] == cond], ax=ax, show=False, return_fig=False, color=gene, size=10, use_raw=True)

In [ ]:
sc.settings.verbosity = 0
warnings.filterwarnings('ignore')
fig, ax = plt.subplots(3, 5, figsize=(30,18))
for ct, ax in tqdm(zip(good_cts,np.ravel(ax))):
    rank_adata = acg_t[acg_t.obs['ct1'] == ct].copy()
    sc.tl.rank_genes_groups(rank_adata, groupby='cond', n_genes=20, groups=['G'], reference='C')
    y = range(20)[::-1]
    x = [i[0] for i in rank_adata.uns['rank_genes_groups']['scores'].tolist()]
    txts = [i[0] for i in rank_adata.uns['rank_genes_groups']['names'].tolist()]
    ax.scatter(x, y, s=0)
    for i, txt in enumerate(txts):
        ax.annotate(txt, (x[i], y[i]), rotation=0, size=10)
    ax.set_title(ct)

In [ ]:
gene = 'AHNAK'
fig, ax = plt.subplots(1,2,figsize=(12,5))
for cond, ax in zip(['C','G'], ax):
    ax.set_facecolor('black')
    sc.pl.umap(acg_t[acg_t.obs['cond'] == cond], ax=ax, show=False, return_fig=False, color=gene, size=10, use_raw=True)

In [ ]:
sc.pl.umap(acg_t, color=proteins, size=2, ncols=8)

In [ ]:
sc.settings.verbosity = 0
warnings.filterwarnings('ignore')
sc.tl.rank_genes_groups(acg_t, groupby='leiden', n_genes=40, groups=['18'],reference='6')
warnings.filterwarnings('default')
sc.pl.rank_genes_groups(acg_t, ncols=5, n_genes=20)
sc.settings.verbosity = 4

In [ ]:
[i[0] for i in acg_t.uns['rank_genes_groups']['names']]

In [ ]:
sc.pl.umap(acg_t, color=[i[0] for i in acg_t.uns['rank_genes_groups']['names'][:20]], size=2, ncols=5)

In [ ]:
sc.pl.umap(acg_t, color=[i[0] for i in acg_t.uns['rank_genes_groups']['names'][20:]], size=2, ncols=5)